In [1]:
print("Settings.py should be e.g. cnn_flows_prtrain")
#DESIGN PARAMETERS FOR NEURAL NETWORK

VALIDATION_SPLIT = 0.5 #indicated ratio of training to validation data: 0.2 -> 20% VALIDATION data
RANDOMIZED_SEED = 20
SHUFFLE_DATASET = False

TRAINING_PREPARATION = False

BATCH_SIZE = 1
TIME_STEPS = 40

SLICE_FROM_TIME_STEP = 0 #slices from timestep SLICE_FROM_TIMESTEP to the last one

NR_EPOCHS = 100

LR = 0.0001
print("Hyperparameters defined")
print("Bug with learning rate when sequence length is not the same for each element of the dataset")

Settings.py should be e.g. cnn_flows_prtrain
Hyperparameters defined
Bug with learning rate when sequence length is not the same for each element of the dataset


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import time
import os
import os.path as path
import copy

from gait_analysis import AnnotationsCasia as Annotations
from gait_analysis import CasiaDataset
from gait_analysis.Config import Config
from gait_analysis import Composer
from gait_analysis import WeightWatcher
from gait_analysis import AccuracyTrackerTrainTest

import matplotlib.pyplot as plt

from gait_analysis.Models.ConvLSTMFlow import ConvLSTMFlow

print("done")

ModuleNotFoundError: No module named 'gait_analysis.Models.ConvLSTMFlow'

In [3]:
net = ConvLSTMFlow()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=LR)

# monitor = WeightWatcher(net,['conv5','fc1'])
tt_acc_tracker = AccuracyTrackerTrainTest([0,1,2])

TODO: doesn't x = x.view(BATCH_SIZE,TIME_STEPS*LSTM_HIDDEN_FEATURES) mix up batch size order?
TODO: BATCH_SIZE is currently fixed and one
TODO: Is x_arr in device?
cuda:0


In [4]:
#change configuration in settings.py
c = Config()
c.config['indexing']['grouping'] = 'person_sequence_angle'
# c.config['transformers']['DimensionResize']['dimension'] = TIME_STEPS
# #c.config['indexing']['people selection'] = [1]
# #c.config['indexing']['sequences_selection'] = ['nm-01']
# c.config['pose']['load'] = False
# c.config['flow']['load'] = True
# c.config['heatmaps']['load'] = False
# #c.config['scenes']['sequences'] = ['nm']
# #c.config['scenes']['angles'] = ['108']
# c.config['dataset_output'] = {
# #         'data': ["scenes","flows","heatmaps_LAnkle","heatmaps_RAnkle"],
#         'data': ['flows'],
#         'label': "annotations"}
composer = Composer()
transformer = composer.compose()
dataset = CasiaDataset(transform=transformer)

loading configuration  cnn_flows_crop
[OK]


In [5]:
dataset_size = len(dataset)
indices = list(range(dataset_size))
print("Dataset size:", len(dataset))
print("Indices size:", len(indices))
split = int(np.floor(VALIDATION_SPLIT * dataset_size))
print("Split:", split)
if SHUFFLE_DATASET:
    np.random.seed(RANDOMIZED_SEED)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]
train_sampler = torch.utils.data.SequentialSampler(train_indices)
test_sampler = torch.utils.data.SequentialSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)

Dataset size: 595
Indices size: 595
Split: 119


In [6]:
# #PREPARATION FOR TRAINING
# loss_array = []
# learning_rate_array = []

# print('Start training...')
# print("Expected loss with {} different classes and {} data elements: {}".format(3, len(dataset)-split, (len(dataset)-split)*np.log(3)))
# running_loss = 0.0
# #print("Data set length:", len((train_loader)), "Validation length:", len(test_loader))
# print("Batch size:", BATCH_SIZE)
# print("Evaluating first element...")
# start_time = time.time()
# i, batch = next(iter(enumerate(train_loader)))
# inputs, labels = batch
# data_in = [s.to(device) for s in inputs['flows']]
# # print("Data in original", data_in)


# # print("Proof for normalized data:", data_in[0][0])
# labels = labels.to(device)
# print("Time steps:{}, input sequence length:{}".format(TIME_STEPS,len(data_in)))
# #print("NN input: ",len(flows),len(flows[0]),len(flows[0][0]),len(flows[0][0][0]),len(flows[0][0][0][0]))
# optimizer.zero_grad() 
# outputs = net(data_in)
# print("Expected output format: [BATCH, NR_CLASSES, TIMESTEPS]")
# # print("Output format:", len(outputs), outputs.size())
# print("Expected label format: [BATCH, TIMESTEPS] (with int-label as each element indicating the correct one)")
# # print("Labels:", len(labels), labels.size())
# # print("Slicing loss. Using loss from:",SLICE_FROM_TIME_STEP,"to",TIME_STEPS)
# print("Final label: ",labels[:,SLICE_FROM_TIME_STEP:TIME_STEPS].size())
# print("Final output:", outputs[:,:,SLICE_FROM_TIME_STEP:TIME_STEPS].size())
# # print("Labels content:", labels)
# labels = labels.squeeze(0)
# outputs = outputs.squeeze(0)
# print("Dimensions:","Labels",labels.size(),"Pred",outputs.size())
# # before: loss = criterion(outputs[:,:,SLICE_FROM_TIME_STEP:TIME_STEPS].float(),labels[:,SLICE_FROM_TIME_STEP:TIME_STEPS].long())
# loss = criterion(outputs[SLICE_FROM_TIME_STEP:TIME_STEPS,:].float(),labels[SLICE_FROM_TIME_STEP:TIME_STEPS].long())
# loss.backward() 
# optimizer.step()

# running_loss += loss.data.item()
# elapsed_time = time.time() - start_time;
# loss_array.append(running_loss)
# learning_rate_array.append(LR)
# print("Loss:{}, expected loss:{}".format(running_loss, np.log(3)))
# print("Time needed:{}s".format(elapsed_time))
# print("Expected loss for total training data: ", (len(dataset)-split)*np.log(3))
# print("Expected training time per epoch:{} min".format(elapsed_time* len(train_loader)/60))
# print("Estimated total training time:{} hours".format(elapsed_time* len(train_loader)*NR_EPOCHS/3600))

In [8]:
print('Start training...')

for epoch in range(NR_EPOCHS): 
    start_time = time.time()
    running_loss_train = 0.0
    running_loss_test = 0.0
    for i, batch in enumerate(train_loader):
        inputs, labels = batch
        data_in = [s.to(device) for s in inputs['flows']]
        labels = labels.to(device)
        
#         print("Time steps:{}, input sequence length:{}".format(TIME_STEPS,len(data_in)))
        #print("NN input: ",len(flows),len(flows[0]),len(flows[0][0]),len(flows[0][0][0]),len(flows[0][0][0][0]))
        optimizer.zero_grad() 
        outputs = net(data_in)
        labels = labels.squeeze(0).long()
        outputs = outputs.squeeze(0).float()
        
#         print("Size output",outputs.size(),"and label", labels.size())
#         print(outputs)
        
        loss = criterion(outputs[SLICE_FROM_TIME_STEP:TIME_STEPS,:].float(),labels[SLICE_FROM_TIME_STEP:TIME_STEPS].long())
        loss.backward() 
        optimizer.step()

        running_loss_train += loss.data.item()/((1-VALIDATION_SPLIT)*len(dataset))
        
#         acc_tracker.update_lr(LR)

        prediction = torch.max(outputs,1)[1]
        label_element = labels
    
        tt_acc_tracker.update_acc(prediction,label_element,"TRAIN")
        #         print("Prediction", labels)
#         acc_tracker.update_acc(prediction,labels)
        
    with torch.no_grad():
        for i,batch in enumerate(test_loader):
            inputs, labels = batch
            data_in = [s.to(device) for s in inputs['flows']]
            labels = labels.to(device)

            outputs = net(data_in)
            labels = labels.squeeze(0).long()
            outputs = outputs.squeeze(0).float()
            loss = criterion(outputs[:,SLICE_FROM_TIME_STEP:TIME_STEPS],labels[SLICE_FROM_TIME_STEP:TIME_STEPS])
            running_loss_test += loss.data.item()/(VALIDATION_SPLIT*len(dataset))

            prediction = torch.max(outputs,1)[1]
            label_element = labels

            tt_acc_tracker.update_acc(prediction,label_element,"TEST")
   
    elapsed_time = time.time() - start_time;      
    print("-----------------------------")
    print("Epoch {}: Loss [{},{}], Acc [{},{}] took {}s".format(epoch, running_loss_train,running_loss_test,tt_acc_tracker.get_acc_tot("TRAIN"),tt_acc_tracker.get_acc_tot("TEST"), np.around(elapsed_time,decimals=2)))
    print("Accuracy by class [{},{}] Total elements: [{},{}]".format(tt_acc_tracker.get_acc("TRAIN"),tt_acc_tracker.get_acc("TEST"), tt_acc_tracker.get_labels_distribution("TRAIN"),tt_acc_tracker.get_labels_distribution("TEST")))

    tt_acc_tracker.update_loss(running_loss_train,"TRAIN")
    tt_acc_tracker.update_loss(running_loss_test,"TEST")
    tt_acc_tracker.update_graph()
    tt_acc_tracker.reset_acc_both()
    tt_acc_tracker.update_lr(LR)

print('...Training finished')


Start training...
-----------------------------
Epoch 0: Loss [0.05680645208027092,0.06935679309164025], Acc [0.982,0.972] took 121.17s
Accuracy by class [[0.986, 0.983, 0.976],[0.979, 0.987, 0.951]] Total elements: [[6066, 6090, 6884],[1460, 1591, 1709]]
-----------------------------
Epoch 1: Loss [0.05508002482683343,0.06919843264847618], Acc [0.983,0.972] took 121.42s
Accuracy by class [[0.987, 0.983, 0.979],[0.976, 0.991, 0.952]] Total elements: [[6066, 6090, 6884],[1460, 1591, 1709]]
-----------------------------
Epoch 2: Loss [0.09811886025890078,0.1008537443307768], Acc [0.968,0.957] took 122.01s
Accuracy by class [[0.972, 0.97, 0.963],[0.982, 0.963, 0.931]] Total elements: [[6066, 6090, 6884],[1460, 1591, 1709]]
-----------------------------
Epoch 3: Loss [0.15207881521934463,0.13855161592990412], Acc [0.947,0.947] took 120.78s
Accuracy by class [[0.955, 0.951, 0.937],[0.96, 0.962, 0.92]] Total elements: [[6066, 6090, 6884],[1460, 1591, 1709]]
-----------------------------
Epoc